# 1.데이터 다운로드 및 전처리

## 1) pykrx 모듈 다운로드

In [1]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.3 MB/s eta 0:00:00


## 2) 데이터 불러오기 (5년 치, 10년 치, 50년 치)

### (1) 8개 종목 선택

KB금융	105560 신한지주	055550 하나금융지주	086790 메리츠금융지주	138040 기업은행	024110 미래에셋증권	006800 NH투자증권	005940 삼성증권	016360

In [2]:
# Make code dictionary.
finance_code_dict = dict()
finance_code_list = "KB금융	105560 신한지주	055550 하나금융지주	086790 메리츠금융지주	138040 기업은행	024110 미래에셋증권	006800 NH투자증권	005940 삼성증권	016360".split()
for i in range(8):
  finance_code_dict[finance_code_list[2*i]] = finance_code_list[2*i + 1]

### (2) 데이터 가져오기 함수 정의 (5y, 10y)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykrx import stock

def get_5y_10y(ticker_name):
  ticker_code = finance_code_dict[ticker_name]
  return stock.get_market_ohlcv("20180101", "20221231", ticker_code),\
  stock.get_market_ohlcv("20130101", "20221231", ticker_code)

### (3) 데이터 그리기 함수 정의

In [4]:
import matplotlib
import matplotlib.pyplot as plt

def draw_graph_10y(ticker_name):

  _, df = get_5y_10y(ticker_name)

  # 1 line, 3 graphs

  # graph 1
  plt.subplot(3, 1, 1)
  series = df['종가']
  plt.title(f"{ticker_name} time series")
  plt.spring()
  plt.plot(series)

  # graph 2
  plt.subplot(3, 1, 2)
  plt.title(f"{ticker_name} difference, time series")
  series_diff = series - series.shift(1)
  plt.plot(series_diff)

  # graph 3
  plt.subplot(3, 1, 3)
  plt.title(f"{ticker_name} difference, histogram")
  plt.hist(series_diff)

  plt.tight_layout()

  plt.show()

### (4) train_data, test_data 얻는 함수

In [17]:
def get_10y_data(ticker_name):
  ticker_code = finance_code_dict[ticker_name]
  train_df = stock.get_market_ohlcv("20130101", "20221231", ticker_code)
  test_df = stock.get_market_ohlcv("20230101", "20230630", ticker_code)
  return train_df['종가'], test_df['종가']

## 3) 데이터 정규화(Normalization)

### (2) 10y, 10y differencing

In [ ]:
# for key in finance_code_dict:
#   draw_graph_10y(key)

# 2.Dataset 윈도우

In [7]:

from torch.utils.data import DataLoader, Dataset
class windowDataset(Dataset):
  # data_stream     : input_window, output_window 크기에 따라 쪼개질 데이터
  # input_window    : 인풋 기간
  # output_window   : 아웃풋 기간
  # stride          :
    def __init__(self, data_stream, input_window=80, output_window=20, stride=5):
        # data_stream의 행 개수를 구한다.
        L = data_stream.shape[0]
        # stride에 따라 샘플 개수를 구한다.
        num_samples = (L - input_window - output_window) // stride + 1

        # [window 크기 * sample 개수] 크기의, 0으로 채워진 배열을 만든다.
        X = np.zeros([input_window, num_samples])
        Y = np.zeros([output_window, num_samples])

        # np.arange(num_samples): range(num_samples) 와 같음
        for i in np.arange(num_samples):
            # 1) X:   input_window 만큼 자르기 (stride * i ~)
            start_x = stride * i
            X[:,i] = data_stream[start_x:start_x + input_window]
            # 2) Y:   output_window 만큼 자르기 (stride * i + input_window ~)
            start_y = start_x + input_window
            Y[:,i] = data_stream[start_y:start_y + output_window]


        # shape       : [window 크기, sample 개수]
        X = X.reshape(X.shape[0], X.shape[1], 1).transpose((1,0,2))
        Y = Y.reshape(Y.shape[0], Y.shape[1], 1).transpose((1,0,2))
        self.x = X
        self.y = Y

        self.len = len(X)

    def __getitem__(self, i):
        return self.x[i], self.y[i]
    def __len__(self):
        return self.len



# 3.Transformer 모델

In [8]:

from torch import nn
import torch
import math

class TFModel(nn.Module):

# iw/ow:      input window, output window
# d_model:    인풋 개수
# nlayers:    인코더 부분의 인코더 개수
# nhead:      multihead attention 개수

    def __init__(self, iw: int, ow: int, d_model: int, nhead: int, nlayers: int, dropout=0.5):
        super(TFModel, self).__init__()

        # 1개 인코더, 인풋 사이즈가 d_model이고 attention 개수는 nhead
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)

        # stacked 인코더, nlayers 만큼 쌓여있다.
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=nlayers)
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        # 인풋 차원 변환. 1차원 -> d_model//2차워 -> d_model차원
        self.encoder = nn.Sequential(
            nn.Linear(1, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, d_model)
        )

        # 차원 변환. d_model -> d_model//2 -> 1
        self.linear =  nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, 1)
        )

        # 차원 변환. iw -> ow
        self.linear2 = nn.Sequential(
            nn.Linear(iw, (iw+ow)//2),
            nn.ReLU(),
            nn.Linear((iw+ow)//2, ow)
        )

    def generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, srcmask):
        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src.transpose(0,1), srcmask).transpose(0,1)
        output = self.linear(output)[:,:,0]
        output = self.linear2(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask


# 4.학습

- 입출력 윈도우 사이즈
- Learning Rate
- Model
  - layer
  - dropout
  - multihead attention 개수
- Cost Function
- Optimizer




In [18]:
# @title Hyper-parameter
INPUT_WINDOW = 21*14
OUTPUT_WINDOW = 4Model
layer
dropout
multihead attention 개수
Cost Function
Optimizer
Hyper-parameter

train_data, test_data = get_10y_data('KB금융')
train_dataset = windowDataset(train_data, input_window=INPUT_WINDOW, output_window=OUTPUT_WINDOW, stride=1)
train_loader = DataLoader(train_dataset, batch_size=64)     # 64 = 2^6, 512 = 2^9
test_dataset = windowDataset(train_data, input_window=INPUT_WINDOW, output_window=OUTPUT_WINDOW, stride=1)
test_loader = DataLoader(train_dataset, batch_size=64)     # 64 = 2^6, 512 = 2^9


if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

lr = 1e-4
model = TFModel(iw=INPUT_WINDOW, ow=OUTPUT_WINDOW, d_model=512, nhead=8, nlayers=4, dropout=0.1).to(device)
criterion = nn.MSELoss()                                            # MSEloss(): ow 각 요소들의 합
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [20]:
# @title Train
'''
[error]
OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB (GPU 0; 14.75 GiB total capacity; 13.76 GiB already allocated; 70.81 MiB free; 13.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

[solution]
https://discuss.pytorch.kr/t/cuda-out-of-memory/216/6
'''
import torch, gc
gc.collect()
torch.cuda.empty_cache()

# for tqdm
from tqdm import tqdm
epoch = 30
model.train()
progress = tqdm(range(epoch))

for i in progress:
  batchloss = 0.0
  for (inputs, outputs) in train_loader:
    # inputs.shape: [batch_size, iw, 1]
    # outputs.shape: [batch_size, ow, 1]

    # Initialize grad
    optimizer.zero_grad()                                           # zero_grad()로 Torch.Tensor.grad 초기화. 초기화하지 않으면 다음 루프 backward() 시에 간섭함.

    # Forward propagation with masking
    src_mask = model.generate_square_subsequent_mask(inputs.shape[1]).to(device)
    result = model(inputs.float().to(device), src_mask)             # forward

    # Backward propagation
    loss = criterion(result, outputs[:,:,0].float().to(device))     # ?? 64개 중 하나만 loss를 담네?
    loss.backward()                                                 # backward
    optimizer.step()
    batchloss += loss

  print()
  progress.set_description(f"loss: {batchloss.cpu().item() / len(train_loader):0.6f}")

progress.close()

  0%|          | 0/30 [00:00<?, ?it/s]

loss: 2169968278.588235:   3%|▎         | 1/30 [00:24<11:48, 24.43s/it]

loss: 2167077707.294117:   7%|▋         | 2/30 [00:44<10:11, 21.85s/it]

loss: 2156706394.352941:  10%|█         | 3/30 [01:03<09:15, 20.56s/it]

loss: 2128638674.823529:  13%|█▎        | 4/30 [01:22<08:39, 19.98s/it]

...

loss: 85715034.352941:  87%|████████▋ | 26/30 [08:30<01:17, 19.48s/it]

loss: 85695397.647059:  90%|█████████ | 27/30 [08:49<00:58, 19.48s/it]

loss: 85687017.411765:  93%|█████████▎| 28/30 [09:09<00:38, 19.49s/it]

loss: 85665031.529412:  97%|█████████▋| 29/30 [09:28<00:19, 19.50s/it]

loss: 85668171.294118: 100%|██████████| 30/30 [09:48<00:00, 19.60s/it]


## 3) Test

In [23]:
# set evaluation mode
model.eval()

# Initialize correct & total
correct = 0
total = 0

# 기울기 계산을 방지하기 위해 torch.no_grad() 블록 안에서 평가
with torch.no_grad():
  for (inputs, outputs) in tqdm(test_loader, desc="Evaluating"):
    # Forward propagation with masking
    src_mask = model.generate_square_subsequent_mask(inputs.shape[1]).to(device)
    result = model(inputs.float().to(device), src_mask)

    # 상승/하강 예측
    predicted_changes = torch.sign(result[:, -1] - inputs[:, -1, 0].to(device))             # 마지막 예측 값 - 마지막 입력 값
    true_changes = torch.sign(outputs[:, -1, 0].to(device) - inputs[:, -1, 0].to(device))  # 실제 마지막 값 - 마지막 입력 값

    # 예측이 맞는 경우
    correct += (predicted_changes == true_changes).sum().item()
    total += inputs.size(0)

  print()
  progress.set_description(f"current accuracy: {correct/total:0.6f}")

# 정확도 계산
accuracy = correct / total
print(f"\nDirectional Accuracy: {accuracy:.4f}")


Evaluating: 100%|██████████| 34/34 [00:05<00:00,  5.91it/s]



Directional Accuracy: 0.1899
